In [1]:
# Importing required Libraries
from pypdf import PdfReader
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google import genai
import chromadb
import pandas as pd
import os

In [2]:
# splitting text
splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

#Creating a list
doc =[]

#Extracting text from pdf splitting it and storing it in list
reader = PdfReader("test_pdf.pdf")
i=1
for page in reader.pages:
    text = page.extract_text()
    chunks=splitter.split_text(text)
    doc.extend([{"page_number":i, "content":chunk}for chunk in chunks])
    i=i+1

    

In [3]:
#Transforming data
document =[d["content"] for d in doc]
metadata =[ {"page_number":d["page_number"]}  for d in doc]
ids=[f"{i}" for i in range(len(doc))]

#Storing in VectorDB
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("my_collection")
collection.add(
    documents=document,
    metadatas=metadata,
    ids=ids
)


In [ ]:
#Question from ChromaDB
query = "What is the main architecture proposed in this paper?"
result = collection.query(
    n_results=1,
    query_texts=[query]
)

# Calling Gemini and asking it 
gemini = genai.Client(api_key=os.getenv("gemini_key"))
ai_result = gemini.models.generate_content(
    model='gemini-2.0-flash',
    contents=f"Explain {query} in context to {result['documents']}"
    
)


Based on the provided context, the main architecture proposed in the paper is **The Transformer**, which utilizes **stacked self-attention** mechanisms and **point-wise, fully connected feed-forward networks**.

In essence, the paper is introducing or elaborating on the Transformer model architecture, and Figure 1 illustrates this architecture. The key components highlighted are:

*   **Stacked self-attention:** This implies that the model uses multiple layers of self-attention, allowing it to capture complex relationships within the input sequence.
*   **Point-wise, fully connected layers:** After the self-attention layers, the model uses feed-forward networks applied to each position in the sequence independently.

Therefore, the main architecture is the Transformer, characterized by its reliance on stacked self-attention and point-wise fully connected layers.



In [ ]:
#Final Result
print(ai_result.text)

Based on the provided context, the main architecture proposed in the paper is **The Transformer**, which utilizes **stacked self-attention** mechanisms and **point-wise, fully connected feed-forward networks**.

In essence, the paper is introducing or elaborating on the Transformer model architecture, and Figure 1 illustrates this architecture. The key components highlighted are:

*   **Stacked self-attention:** This implies that the model uses multiple layers of self-attention, allowing it to capture complex relationships within the input sequence.
*   **Point-wise, fully connected layers:** After the self-attention layers, the model uses feed-forward networks applied to each position in the sequence independently.

Therefore, the main architecture is the Transformer, characterized by its reliance on stacked self-attention and point-wise fully connected layers.

